In [27]:
import torch
import torch.nn as nn 
import torch.optim as optim 
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

In [31]:
digits = load_digits()
X = digits.data
y = digits.target

In [38]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y)

In [41]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [42]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [44]:
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(64, 128)
        self.batchnorm1 = nn.BatchNorm1d(128) # Apply BatchNorm on layer 1 output
        self.fc2 = nn.Linear(128, 64)
        self.batchnorm2 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, 10) #output size for 10 classes

    def forward(self, x):
        x = torch.relu(self.batchnorm1(self.fc1(x)))
        x = torch.relu(self.batchnorm2(self.fc2(x)))
        x = self.fc3(x) # No activation or batchnorm on the final layer (for classification)

        return x

model = NN()
criterion = nn.CrossEntropyLoss() #Sparse Cross-Entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    _, predicted = torch.max(test_outputs, 1)
    accuracy = (predicted == y_test_tensor).float().mean()
    print(f'Test Accuracy: {accuracy:.4f}')


Epoch [1/10], Loss: 1.6360
Epoch [2/10], Loss: 0.6902
Epoch [3/10], Loss: 1.2879
Epoch [4/10], Loss: 0.4222
Epoch [5/10], Loss: 0.5763
Epoch [6/10], Loss: 1.9255
Epoch [7/10], Loss: 0.2558
Epoch [8/10], Loss: 0.2967
Epoch [9/10], Loss: 0.4225
Epoch [10/10], Loss: 1.1459
Test Accuracy: 0.9733
